<a href="https://colab.research.google.com/github/internship-bvoc2020/MachineLearning/blob/master/ML_ModelBuilding_playbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Supervised Machine Learning 

This set of Machine Learning problems have a Target Label set present , using which Algorithm learns patterns on Training dataset and then Predicts on Test/Validation set

The set of features which are fed into the model are called *Independent variables* .The Target Label which needs to be predicted is called *Dependant Variable*

Supervised are usually classified into two kind of problem categories -

**Classification** - Where Target variable is a categorical one either being Binary (1/0 , Yes/No) or MultiClass [0,1,2] , [Yes/No/Maybe] 

**Regression** - Where Target variable is a real value or a probability that needs to be predicted e.g - Students final score 

Algorithms used for solving these problems are of the following kind -

1. Linear Models - includes Linear Regression , Logistic Regression (Classification) , Lasso Regression etc

2. Kernel Models - Support Vector Machine ,Principal Component Analysis etc

3. Trees based Models - Decision Trees , Random Forest , Gradient Boosting ,Bagging techniques

4. Neural Networks

### Unsupervised Machine Learning

This involves drawing inferences from datasets consisting of input data without labeled (Target Label) responses.

Techniques used over here are Clustering approaches (KNN, Hierarchical ,DBScan etc)  and Association Rules



### Semi Supervised Machine Learning

This branch deals with problems having datasets which are labelled in small proportion.Requires combination of bother supervised & unsupervised techniques in solving the problem

Few techniques used are :  Label Propagation , Active Learning etc

**Supervised Machine Learn - Classification Modeling**

Build a model to predict which Loan Applications will be Accepted or Rejected . 

*'Loan_Status*' in the below dataset defines the application being accepted or not based on Applicants Demographic details captured in the dataset 

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import *
from sklearn.preprocessing import *
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
df_data = pd.read_csv("dataset_model.csv",sep=",")
df_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [0]:
df_data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [0]:
df_data.shape

(614, 13)

In [0]:
df_data['Loan_Status'].unique()

array(['Y', 'N'], dtype=object)


### Identifying Potential Features in a dataset

When given a dataset , here a labelled one with Laon_Status being the Target Label/Dependent Label , rest of the fields become potential Independent Labels or Features.

From the "potenital" list of features , we first check if any Primary Key record identification fields are present which were created only for Identification purposes.Such fields are removed from Features list as they have no usage in predicting Target Label.Ex - Aadhar card , PAN no , Name etc 

Here 'Loan_ID' seems to be a unique record identification field , which doesn't seem to capture any information about the applicant.


In [0]:
#Checking if Loan ID has all unique values , and thus can be removed 
len(df_data['Loan_ID'].unique())

614

In [0]:
#Saving the Loan ID in the sequence of application in another dataframe
df_id = df_data['Loan_ID']
#Dropping Loan ID from Featureset
df_data=df_data.drop(columns='Loan_ID')
df_data.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

Target variables in Classification problems , if they're not numeric then they are always encoded using Label Encoder. This is to ensure that either the Binary Class(0/1) or Multi-Class (0/1/2..) is clearly understood by Model and use this information in Featureset to perform prediction.

Also , if there are records which have Target Variables as missing , then they are  removed from Training set . No missing values treatment is done 


### Performing EDA & Feature Engineering on dataset
